# The goal of this file is to test PFGAP on Graphs

In [1]:
import numpy as np
import networkx as nx
from tqdm.notebook import tqdm  # For a nice progress bar

# Scikit-learn imports
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# PyTorch Geometric imports
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import to_networkx

# --- Load Dataset ---
print("Loading PROTEINS dataset...")
dataset = TUDataset(root='data/TUDataset', name='PROTEINS')
print("Dataset loaded successfully.")
print("-" * 30)

# --- Your Distance Function ---
def graph_distance(graph1, graph2):
    """Calculates distance based on Laplacian spectra."""
    # Convert PyG graphs to NetworkX graphs
    # node_attrs=None and edge_attrs=None can speed this up
    g1_nx = to_networkx(graph1, node_attrs=None, edge_attrs=None)
    g2_nx = to_networkx(graph2, node_attrs=None, edge_attrs=None)
    
    spec1 = nx.laplacian_spectrum(g1_nx)
    spec2 = nx.laplacian_spectrum(g2_nx)
    
    # Use the smaller of the two spectrum lengths for comparison
    k = min(len(spec1), len(spec2))
    
    # Calculate the L2 norm (Euclidean distance) of the spectra
    return np.linalg.norm(spec1[:k] - spec2[:k])

Loading PROTEINS dataset...
Dataset loaded successfully.
------------------------------


In [2]:
# Get the total number of graphs in the dataset
print(f"Total number of graphs in dataset: {len(dataset)}")

# Sample 100 random indices without replacement
np.random.seed(42)  # For reproducibility
sample_indices = np.random.choice(len(dataset), size=500, replace=False)

# Create a sample dataset (list of sampled graphs)
dataset = [dataset[i] for i in sample_indices]

print(f"Sampled {len(dataset)} graphs from the dataset")

# Show information about the first graph in the sample
first_graph = dataset[0]
print(f"\nFirst graph in sample:")
print(f"- Number of nodes: {first_graph.num_nodes}")
print(f"- Number of edges: {first_graph.num_edges}")
print(f"- Label: {first_graph.y.item()}")

Total number of graphs in dataset: 1113
Sampled 500 graphs from the dataset

First graph in sample:
- Number of nodes: 5
- Number of edges: 14
- Label: 1


In [3]:
num_graphs = len(dataset)
# Initialize an empty square matrix to hold the distances
distance_matrix = np.zeros((num_graphs, num_graphs))

print(f"Calculating {num_graphs}x{num_graphs} distance matrix...")

# Use tqdm for a progress bar
for i in tqdm(range(num_graphs)):
    for j in range(i, num_graphs): # We only need to compute the upper triangle
        if i == j:
            continue # Distance to self is 0
        
        # Calculate and store the distance
        dist = graph_distance(dataset[i], dataset[j])
        distance_matrix[i, j] = dist
        distance_matrix[j, i] = dist # The matrix is symmetric

print("\nDistance matrix calculation complete!")
print(f"Shape of distance matrix: {distance_matrix.shape}")

Calculating 500x500 distance matrix...


  0%|          | 0/500 [00:00<?, ?it/s]


Distance matrix calculation complete!
Shape of distance matrix: (500, 500)


In [6]:
# Get the labels for each graph
labels = np.array([graph.y.item() for graph in dataset])

# Create an array of indices [0, 1, 2, ..., n-1]
indices = np.arange(num_graphs)

# Split indices and labels into training and testing sets
# We stratify by labels to ensure both sets have a similar class distribution
train_indices, test_indices, y_train, y_test = train_test_split(
    indices, labels, test_size=0.3, random_state=42, stratify=labels
)

# Now, create the training and testing distance matrices
# X_train should contain distances between all training samples
X_train_precomputed = distance_matrix[train_indices, :][:, train_indices]

# X_test should contain distances between test samples (rows) and training samples (columns)
X_test_precomputed = distance_matrix[test_indices, :][:, train_indices]


print(f"Shape of training distance matrix: {X_train_precomputed.shape}")
print(f"Shape of training labels: {y_train.shape}")
print(f"Shape of testing distance matrix: {X_test_precomputed.shape}")
print(f"Shape of testing labels: {y_test.shape}")

Shape of training distance matrix: (350, 350)
Shape of training labels: (350,)
Shape of testing distance matrix: (150, 350)
Shape of testing labels: (150,)


In [7]:
# Initialize the classifier with k=5 neighbors
# IMPORTANT: We set metric='precomputed'
knn = KNeighborsClassifier(n_neighbors=5, metric='precomputed')

# Train the model
print("Training KNN classifier...")
knn.fit(X_train_precomputed, y_train)

# Make predictions on the test set
print("Making predictions...")
y_pred = knn.predict(X_test_precomputed)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("-" * 30)
print(f"✅ Model Accuracy: {accuracy * 100:.2f}%")

Training KNN classifier...
Making predictions...
------------------------------
✅ Model Accuracy: 66.67%


# Visualizations